# Chapter 8 Pythonic Productivity Techniques

# 8.1 Exploring Python Modules and Objects
In this chapter you’ll learn two simple techniques you can use to explore Python classes and methods interactively from the interpreter.

想像一下你正在使用 datetime module，但又不知道有哪些功能可使用  
你可以查詢官方文檔或是使用`dir()` 來查看

In [13]:
import datetime

# dir() gives you a quick overview of what’s available
# on a module or class.
dir(datetime)

['MAXYEAR',
 'MINYEAR',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'date',
 'datetime',
 'datetime_CAPI',
 'sys',
 'time',
 'timedelta',
 'timezone',
 'tzinfo']

但有時不需要看到這麼多資訊，能用以下方法篩選

In [14]:
[_ for _ in dir(datetime) if 'date' in _.lower()]

['date', 'datetime', 'datetime_CAPI']

呼叫 `help()` 能看到詳細的說明文件

In [15]:
# help(datetime)

**Key Takeaways**
+ Use the built-in dir() function to interactively explore Python
modules and classes from an interpreter session.
+ The help() built-in lets you browse through the documentation right from your interpreter (hit q to exit.)

# 8.2 Isolating Project Dependencies With Virtualenv
使用 pip 來安裝套件，預設安裝在 global 環境，如果套件多了起來那將會相當麻煩


## Virtual Environments to the Rescue
問題的解法將會是使用 virtual environments，這讓你能獨立出一個環境來安裝套件  
  
virtual environments 環境內的套件未必有實體檔案，多數時候是指向 global 的套件實體

先來檢查 global 環境  
  
```
$ which pip3
/usr/local/bin/pip3
```
  
建立 python virtual environment  
  
```
$ python3 -m venv ./venv
bin include lib pyvenv.cfg
```
  
venv 底下的內容
```
$ ls venv/

```
  
如果這時候檢查 pip3 在哪，仍然是在 global 環境，所以我們要啟動虛擬環境  
進入虛擬環境後，pip3 位置就會變成當前資料夾，當前環境是沒有任何套件的
```
$ source ./venv/bin/activate
$ which pip3
/Users/dan/my-project/venv/bin/pip3
$ which python
/Users/dan/my-project/venv/bin/python
```
離開環境的指令  
```
$ deactivate
```


**Key Takeaways**
+ Virtual environments keep your project dependencies separated. They help you avoid version conflicts between packages
and different versions of the Python runtime.
+ As a best practice, all of your Python projects should use virtual
environments to store their dependencies. This will help avoid
headaches.

# 8.3 Peeking Behind the Bytecode Curtain
CPython interpreter 在執行程式時，會先把程式碼轉為 bytecode   
  
這種方式比起直接執行程式更有效率，原因是可以把一些複雜的 bytecode 快取起來，存放到 .pyc 和 .pyo 檔案，下此取用就會更快  
  
稍微了解一下 CPython 的運作機制能幫助寫出更有效率的程式碼

In [16]:
def greet(name):
    return 'Hello, ' + name + '!'

可以用 `__code__` attribute 來觀看轉成 bytecode 的樣子
+ co_code 是 function 本身
+ 變數和常數則放在 co_consts 和 co_varnames

In [17]:
greet.__code__.co_code

b'd\x01|\x00\x17\x00d\x02\x17\x00S\x00'

In [18]:
greet.__code__.co_consts

(None, 'Hello, ', '!')

In [19]:
greet.__code__.co_varnames

('name',)

但 co_code 不是給人類讀的，需要用 disassembler 來查看實際執行的 opcode  

CPython’s virtual machine is an implementation of such a stack machine  
1. 取得 co_consts[1] = 'Hello, ' 放入 stack  
2. 接著取出 name 一樣放入 stack  
3. 執行 BINARY_ADD 把 stack 最上面兩個元素相加後放回 stack

In [20]:
import dis
dis.dis(greet)

  2           0 LOAD_CONST               1 ('Hello, ')
              2 LOAD_FAST                0 (name)
              4 BINARY_ADD
              6 LOAD_CONST               2 ('!')
              8 BINARY_ADD
             10 RETURN_VALUE


**Key Takeaways**
+ CPython executes programs by first translating them into an intermediate bytecode and then running the bytecode on a stackbased virtual machine.
+ You can use the built-in dis module to peek behind the scenes
and inspect the bytecode.
+ Study up on virtual machines—it’s worth it.